In [1]:
import os, time, json
import regex as re
import pandas as pd
from dotenv import load_dotenv
from industry_grouping import startup_ind_main
from read_companies import batch_read_companies
from read_deals import get_deals_by_pipeline, list_deal_associations
from bs4 import BeautifulSoup
import requests

load_dotenv()

DBDIR = "C:/Users/galon/Sputnik ATX Team Dropbox/Programming Datasets"
RAW_DIR = os.path.join(DBDIR, "data", "raw")
CLEAN_DIR = os.path.join(DBDIR, "data", "clean")
IND_DIR = os.path.join(CLEAN_DIR, "industry_mapping")


VC_INDUSTRY_COLS = os.path.join(RAW_DIR, "hs", "vc_industry_columns.csv")
STARTPATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase', 'cb_startups_main.csv')
INVPATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase', 'cb_vc_main_merged.csv')
MAPPING_OUTPATH = os.path.join(IND_DIR, "cb_starts_mapped.csv")
START_HERF_PATH = os.path.join(CLEAN_DIR, 'scraped_data', 'crunchbase','cb_inv_overview_scraped.csv')
VC_PATH = os.path.join(RAW_DIR, 'hs', 'vc_list_export.csv')
# propose 397 each for first citizens

In [2]:
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', 1000)
# pd.set_option('display.max_rows', 50)
# df = startup_ind_main()
# def company_investments(df, id_, arg_list):

#     print(len(df[df['id']==id_]))
#     df = df[df['pf_inds'].str.contains('|'.join(arg_list))]
#     return df[df['id']==id_]

# def prelim(df):
        
#     num_specific = df.loc[df['pf_inds'].apply(lambda x: len(re.findall(';',str(x)))) == 1, :]
#     ind_specific = df.loc[df['pf_inds'] == ';Mental Health', :]
#     return ind_specific

# company_investments(df, 9060201392, ['Fintech', 'Employment', 'Human Resources'])

In [25]:
# dict_list = get_deals_by_pipeline([], 'default')
def get_company_associations(dict_list):
    
    for dictionary in dict_list:
        
        dictionary['associated_companies'] = [result.id for result in list_deal_associations(dictionary['hs_object_id'], 'company').results]
    
    return pd.DataFrame.from_dict(dict_list).explode('associated_companies').drop(['hs_lastmodifieddate', 'createdate'], axis=1).rename(columns={'hs_object_id':'deal_id'})
    

In [26]:
def get_contact_associations(dlist):
    
    for dictionary in dlist:
        
        dictionary['associated_contacts'] = [result.id for result in list_deal_associations(dictionary['hs_object_id'], 'contact').results]
        
    return pd.DataFrame.from_dict(dlist).explode('associated_contacts').drop(['hs_lastmodifieddate', 'createdate'], axis=1).rename(columns={'hs_object_id':'deal_id'})


In [32]:
def merge_associations(dict_list, *args):
    
    for arg in args:
        
        if arg == 'company':
            
            df = get_company_associations(dict_list)
            company_inputs = [{'id' : str(id_)} for id_ in list(df['associated_companies'].values)]
            companies = batch_read_companies(['name', 'id'], [], None, company_inputs).results
            company_list = [company.properties for company in companies]
            company_df = pd.DataFrame.from_dict(company_list).drop(['hs_lastmodifieddate','createdate'],axis=1).rename(columns={'hs_object_id':'company_id'})
            df = df.merge(company_df, how='left', left_on='associated_companies', right_on='company_id')
        if arg == 'contact':
            
            df = get_contact_associations(dict_list)
            contact_inputs = [{'id' : str(id_)} for id_ in list(df['associated_contacts'].values)]
            contacts = batch_read_contacts(['firstname', 'lastname','email', 'id'], [], None, contact_inputs).results
            contact_list = [contact.properties for contact in contacts]
            contact_df = pd.DataFrame.from_dict(contact_list).drop(['hs_lastmodifieddate','createdate'],axis=1).rename(columns={'hs_object_id':'contact_id'})
            df = df.merge(contact_df, how='left', left_on='associated_contacts', right_on='contact_id')
            
    return df

In [33]:
merge_associations(get_deals_by_pipeline([], 'default'), 'company', 'contact')

company
contact


NameError: name 'batch_read_contacts' is not defined